In [1]:
import numpy as np

from plotly.graph_objects import Scatter, FigureWidget, Layout
from ipywidgets import IntSlider, FloatSlider, HBox, VBox

In [2]:
def tiro(m, t):
    y = m[0] + m[1] * t - (1/2) * m[2] * t**2
    return y

In [3]:
# crear datos sintéticos:
Ni = 11
tobs = np.linspace(0, 2, Ni)
mreal = np.array([2., 10., 9.8])

sigmai = 0.2
ruidoi = np.random.randn(len(tobs))
yreal = tiro(mreal, tobs)
yobsi = yreal + sigmai*ruidoi

t = np.linspace(0, 2, 101)

# matriz G:
G = np.column_stack([np.ones(Ni), tobs, -0.5*tobs**2])

mi = np.linalg.solve(G.T @ G, G.T @ yobsi)

# Simulacion para inversion inicial:
yi = tiro(mi, t)

pi = np.random.permutation(Ni)

In [4]:
sigma = FloatSlider(
    value=sigmai,
    min=0.1,
    max=0.4,
    step=0.02,
    description='sigma:',
    continuous_update=False
)
N = IntSlider(
    value=Ni,
    min=3,
    max=11,
    step=1,
    description='N:',
    continuous_update=False
)

In [5]:
container = HBox(children=[sigma, N])

In [23]:
# Assign an empty figure widget with two traces
trace0 = Scatter(x=tobs, y=yreal, opacity=0.5, mode='lines', line=dict(color="#000000", dash='dash'), name='real')
trace1 = Scatter(x=tobs, y=yobsi, opacity=0.75, name='observado')
trace2 = Scatter(x=t, y=yi, opacity=0.75, name='modelado')
g = FigureWidget(data=[trace0, trace1, trace2],
                    layout=Layout(
                        title=dict(
                            text='Tiro parabolico (minimos cuadrados)'
                        ),
                        yaxis_range = [0,8],
                        height = 600,
                        width = 600
                    ))

In [24]:
def response(change):
    Ns = N.value
    sigmas = sigma.value
    pio = np.sort(pi[:Ns])
    tobss = tobs[pio]
    yobss = yreal[pio] + sigmas*ruidoi[pio]
    G = np.column_stack([np.ones(Ns), tobss, -0.5*tobss**2])
    mest = np.linalg.solve(G.T @ G, G.T @ yobss)
    yi = tiro(mest,t)
    with g.batch_update():
        g.data[1].x = tobss
        g.data[1].y = yobss
        g.data[2].y = yi

In [25]:
sigma.observe(response, names="value")
N.observe(response, names="value")

In [26]:
VBox([container,g])